In [1]:
import tensorflow as tf

import os
import numpy as np
#os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd

import MBQuantSimul

In [2]:
def BasicBlock_MBQuantSimul(x, channel_size, name, strides=1, kernel_init="glorot_uniform", kernel_reg = None, tau=1.0):
    x_1 = x
    x_2 = x
    
    if strides != 1:
        #option_a
        x_2 = tf.keras.layers.MaxPool2D(1, strides=strides, padding="same", name=name+"_sc_maxpool"  )( tf.pad(x_2, ((0,0), (0,0), (0,0), (0, channel_size-x_2.shape[-1])), name=name+"_sc_optionA") )
        #option_b
        #x_2 = tf.keras.layers.Conv2D(channel_size, 1, strides=strides, padding='same', use_bias=False, name=name+"_sc_conv", kernel_initializer=INIT)(x_2)
        #x_2 = tf.keras.layers.BatchNormalization(name=name+"_sc_bn")(x_2)

    x_1 = MBQuantSimul.MBQuantSimulConv2D(channel_size, 3, tau=tau, strides=strides, padding="same", use_bias=False, name=name+"_conv1", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x_1)
    x_1 = tf.keras.layers.BatchNormalization(name=name+"_bn1")(x_1)
    x_1 = tf.keras.layers.Activation('relu', name=name+"_act1")(x_1)
    x_1 = MBQuantSimul.MBQuantActivation(tau=tau, name=name+"_act1_quantized")(x_1)
    
    x_1 = MBQuantSimul.MBQuantSimulConv2D(channel_size, 3, tau=tau, strides=1, padding="same", use_bias=False, name=name+"_conv2", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x_1)
    x_1 = tf.keras.layers.BatchNormalization(name=name+"_bn2")(x_1)
    
    x = tf.keras.layers.Add(name=name+"_add")([x_1, x_2])
    x = tf.keras.layers.Activation('relu', name=name+"_act2")(x)
    x = MBQuantSimul.MBQuantActivation(tau=tau, name=name+"_act2_quantized")(x)

    return x

def ResNet20_MBQuantSimul(input_shape=(32,32,3), classes=10, channel_sizes=16, kernel_init="glorot_uniform", kernel_reg=tf.keras.regularizers.L2(1e-4)):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    
    #pre
    x = MBQuantSimul.MBQuantSimulConv2D(channel_sizes, 3, tau=1.0, qconfig="distribution_aware", strides=1, padding="same", use_bias=False, name="pre_conv", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x)
    x = tf.keras.layers.BatchNormalization(name="pre_bn")(x)
    x = tf.keras.layers.Activation("relu",name="pre_act")(x)
    x = MBQuantSimul.MBQuantActivation(tau=25.0, name="pre_act_quantized")(x)

    #blocks_1
    x = BasicBlock_MBQuantSimul(x, channel_sizes, "blocks_1_1", strides=1, tau=1.0 )
    for i in range(1,3):
        x = BasicBlock_MBQuantSimul(x, channel_sizes, "blocks_1_"+str(i+1), strides=1, tau=1.0)

    #blocks_2
    x = BasicBlock_MBQuantSimul(x, channel_sizes*2, "blocks_2_1", strides=2, tau=1.0)
    for i in range(1,3):
        x = BasicBlock_MBQuantSimul(x, channel_sizes*2, "blocks_2_"+str(i+1), strides=1, tau=1.0)

    #blocks_3
    x = BasicBlock_MBQuantSimul(x, channel_sizes*4, "blocks_3_1", strides=2, tau=1.0)
    for i in range(1,3):
        x = BasicBlock_MBQuantSimul(x, channel_sizes*4, "blocks_3_"+str(i+1), strides=1, tau=1.0)
    
    #pred
    x = tf.keras.layers.GlobalAveragePooling2D(name="pred_gap")(x)
    x = MBQuantSimul.MBQuantDense(classes, name="pred_dense", tau=1.0, qconfig="distribution_aware", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x) #BIAS REGULARIZER X
    #x = tf.keras.layers.Dense(classes, name="pred_dense", kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(x) #BIAS REGULARIZER X
    x = tf.keras.layers.Activation("softmax", name="pred_out")(x)
    outputs = MBQuantSimul.MBQuantActivation(tau=1.0, name="pred_out_quantized")(x)
    
    return MBQuantSimul.MBQuantModel(inputs=inputs, outputs=outputs, name="ResNet20")
    #return tf.keras.Model(inputs=inputs, outputs=outputs, name="ResNet20")

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = tf.keras.utils.to_categorical(y_train)# tf.squeeze( tf.one_hot( y_train, y_train.max()+1 ) )
y_test = tf.keras.utils.to_categorical(y_test) #tf.squeeze( tf.one_hot( y_test, y_test.max()+1 ) )

x_train = x_train/255.0
x_test = x_test/255.0
x_mean = np.mean(x_train, axis=(0,1,2), keepdims=True)
x_std = np.std(x_train, axis=(0,1,2), keepdims=True)

x_train = (x_train - x_mean)/x_std
x_test = (x_test - x_mean)/x_std

#x_train = tf.cast(x_train, tf.float32)
#x_test = tf.cast(x_test, tf.float32)
#y_train = tf.cast(y_train, tf.float32)
#y_test = tf.cast(y_test, tf.float32)

In [4]:
def scheduler_200(epoch, lr):
    if epoch <60:
        return 0.1
    elif epoch <120:
        return 0.02
    elif epoch <160:
        return 0.004
    else:
        return 0.0008
LR_Scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler_200)

In [5]:
savename = "MBQuant_ResNet20_CIFAR10"
checkpoint_filepath = './' + savename + '/checkpoint_scratch-{epoch}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,save_best_only=False)

In [6]:
model = ResNet20_MBQuantSimul()
model.accumulate_grads = False

In [ ]:
optim = tf.keras.optimizers.SGD(0.1, momentum=0.9, nesterov=True)
loss_f = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model.compile(optimizer=optim, loss=loss_f, metrics=['accuracy'])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True)
datagen.fit(x_train)

history = model.fit(datagen.flow(x_train, y_train, batch_size=128), validation_data=(x_test, y_test), epochs=200, callbacks=[model_checkpoint_callback, LR_Scheduler])

hist_df = pd.DataFrame(history.history)
hist_df.to_csv('MBQuantSimul_ResNet20_CIFAR10.csv'.format(savename), index=False)

#pd.read_csv("BASELINE_ResNet20_CIFAR10.csv")

Epoch 1/200
391/391 [==============================] - 46s 91ms/step - loss: 7.7088 - accuracy: 0.2625 - val_loss: 9.6621 - val_accuracy: 0.3050 - lr: 0.1000
Epoch 2/200
391/391 [==============================] - 35s 88ms/step - loss: 10.1473 - accuracy: 0.3020 - val_loss: 11.5403 - val_accuracy: 0.2257 - lr: 0.1000
Epoch 3/200
391/391 [==============================] - 34s 87ms/step - loss: 10.1585 - accuracy: 0.3192 - val_loss: 10.5262 - val_accuracy: 0.3180 - lr: 0.1000
Epoch 4/200
391/391 [==============================] - 32s 83ms/step - loss: 10.1528 - accuracy: 0.3264 - val_loss: 11.4463 - val_accuracy: 0.2670 - lr: 0.1000
Epoch 5/200
391/391 [==============================] - 33s 84ms/step - loss: 9.9882 - accuracy: 0.3382 - val_loss: 11.9600 - val_accuracy: 0.2401 - lr: 0.1000
Epoch 6/200
391/391 [==============================] - 35s 89ms/step - loss: 9.8931 - accuracy: 0.3511 - val_loss: 10.5594 - val_accuracy: 0.3268 - lr: 0.1000
Epoch 7/200
391/391 [=======================